<a href="https://colab.research.google.com/github/cmajorros/convert_pandas_to_tableau_hyper/blob/main/convert_panda_to_hyper_and_publish_to_tableauserver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use case description**

1) Combine different datasource and calculation from multiple sources e.g MySQL database , csv , hadoop etc and perform some calculation using python

2) Convert the output dataframe to Tableau hyperfile using library pantab.

3) Publish the hyperfile to Tableau Server.

**Step 1**

Import all library and setting up the environment. 

  - To authenticate tableau server you need tableau serverclient to be installed 
  - Pantab for converting pandas to hyper 


In [ ]:
!pip install pantab
from tableauhyperapi import HyperProcess, Telemetry, Connection, CreateMode, NOT_NULLABLE, NULLABLE, SqlType, \
TableDefinition, Inserter, escape_name, escape_string_literal, HyperException, TableName
from time import time
import numpy as np
import pandas as pd
import os
import mysql.connector as sql
import pantab
import tableauserverclient as TSC

In [ ]:
Time = pd.read_csv("time.csv",dtype={'Hour': int})
print(Time)

    Hour
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23


In [ ]:
data_hyper = time

**Step 2:** 
Convert dataframe to hyper file and name as example.hyper

In [ ]:
pantab.frame_to_hyper(data_hyper, "example.hyper", table="extract")

**Step 3:**

-  Login to Tableau server. In this scenario I will login to "Default" site by using fake username and ID. 
-  To make it more secure and preventing from password changing when you deploy the code. You can apply for token on Tableau server and use the token to authenticate.



In [ ]:
# Don't recommend saving your credentials in the script, store elsewhere
tableau_auth = TSC.TableauAuth('admin',
                               '111111', site_id = 'Default')
server = TSC.Server('http://uat.tableau.yourcompanyname.com',
                    use_server_version=False)

**step 4 : publishing the datasource**
-  To publish the datasource I will overwrite the existing datasource (if any)
-  You need to know the datasource ID before publishing. This can be checked on tableau server or Workgroup database.
-  After publishing, we will signout from the server


In [ ]:

#If Datasource with the same name exist it will overwrite.
# Put the projectid you just found here, and name the datasource you want to publish
mydatasourceitem = TSC.DatasourceItem('6',
                                      name='example')
item = server.datasources.publish(mydatasourceitem,"/example.hyper", 'Overwrite')
print("{} published with id: {}".format(item.name, item.id))

server.auth.sign_out()